In [1]:
import io
import pandas as pd
import requests
from requests import post, get
import base64
import json

import random
from time import sleep

In [6]:
df = pd.read_csv('Global_charts.csv')
df = df.drop(columns=['missingRequiredFields', 'chartEntryData.rankingMetric.type', 'trackMetadata.artists', 'trackMetadata.producers', 'trackMetadata.labels', 'trackMetadata.songWriters', 'externalUrl'])
df = df.rename(columns={"chartEntryData.currentRank": "rank", "chartEntryData.previousRank": "prevRank", "chartEntryData.peakRank": "peakRank", "chartEntryData.peakDate": "peakDate", "chartEntryData.appearancesOnChart": "weeksOnChart", "chartEntryData.consecutiveAppearancesOnChart": "streak", "chartEntryData.rankingMetric.value": "streams", "chartEntryData.entryStatus": "trend", "chartEntryData.entryRank": "entryRank", "chartEntryData.entryDate": "entryDate", "trackMetadata.trackName": "trackName", "trackMetadata.trackUri": "track_ID", "trackMetadata.displayImageUri": "image", "trackMetadata.releaseDate": "releaseDate", "name": "artist", "spotifyUri": "artist_ID"})


df["track_ID"] = df["track_ID"].str.replace('spotify:track:', '')
df["artist_ID"] = df["artist_ID"].str.replace('spotify:artist:', '')

df

,rank,prevRank,peakRank,peakDate,weeksOnChart,streak,streams,trend,entryRank,entryDate,trackName,track_ID,image,releaseDate,artist,artist_ID,week
0,1,1,1,2024-02-15,5,5,41483503,NO_CHANGE,24,2024-01-25,Beautiful Things,6tNQ70jh4OwmPGpYy6R2o9,https://i.scdn.co/image/ab67616d00001e02f2a919...,2024-01-18,Benson Boone,22wbnEMDvgVIAGdFeek6ET,2024-02-22
1,2,31,2,2024-02-22,2,2,36879384,MOVED_UP,31,2024-02-15,TEXAS HOLD 'EM,7AYmToZ5y67fBjq4RLFbuq,https://i.scdn.co/image/ab67616d00001e026c1836...,2024-02-10,Beyoncé,6vWDO969PvNqNYHIOW5v0m,2024-02-22
2,3,4,3,2024-02-22,2,2,34127981,MOVED_UP,4,2024-02-15,CARNIVAL,3w0w2T288dec0mgeZZqoNN,https://i.scdn.co/image/ab67616d00001e020a31b4...,2024-02-09,¥$,4xPQFgDA5M2xa0ZGo5iIsv,2024-02-22
3,4,5,2,2023-07-06,55,49,33526124,MOVED_UP,16,2019-08-29,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,https://i.scdn.co/image/ab67616d00001e02e787cf...,2019-08-23,Taylor Swift,06HL4z0CvFAxyc27GXpf02,2024-02-22
4,5,3,1,2023-11-16,23,23,33039462,MOVED_DOWN,14,2023-09-21,greedy,3rUGC1vUpkDG9CZFHMur1t,https://i.scdn.co/image/ab67616d00001e0222fd80...,2023-09-15,Tate McRae,45dkTj5sMRSjrmBSBeiHym,2024-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31195,196,197,5,2020-02-20,54,2,4994138,MOVED_UP,7,2020-02-13,Intentions (feat. Quavo),4umIPjkehX1r7uhmGvXiSV,https://i.scdn.co/image/ab67616d00001e027fe4a8...,2020-02-14,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,2021-03-04
31196,197,192,49,2020-11-19,22,22,4982892,MOVED_DOWN,133,2020-10-08,Sofia,7B3z0ySL9Rr0XvZEAjWZzM,https://i.scdn.co/image/ab67616d00001e0233ccb6...,2019-08-02,Clairo,3l0CmX0FuQjFxr8SK7Vqag,2021-03-04
31197,198,196,41,2017-01-12,149,3,4974880,MOVED_DOWN,123,2016-12-29,Photograph,6fxVffaTuwjgEk5h9QyRjy,https://i.scdn.co/image/ab67616d00001e0213b3e3...,2014-06-21,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,2021-03-04
31198,199,-1,2,2017-03-02,161,1,4962976,RE_ENTRY,132,2017-02-23,Something Just Like This,6RUKPb4LETWmmr3iAEQktW,https://i.scdn.co/image/ab67616d00001e020c13d3...,2017-04-07,The Chainsmokers,69GGBxA162lTqCwzJG5jLp,2021-03-04


## <u>API Credentials and Access token </U>

In [7]:
# Your Spotify API Credentials
client_id = 'a6fc0e1560134b0bbab0159d5f6f1566'
client_secret = 'c7c61b113b754dce800ee715fcb84953'

In [8]:
# Function to get Spotify access token
def get_token():
	auth_string = client_id + ":" + client_secret
	auth_bytes = auth_string.encode("utf-8")
	auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

	url = "https://accounts.spotify.com/api/token"
	headers = {
		"Authorization": "Basic " + auth_base64,
		"Content-Type": "application/x-www-form-urlencoded"
	}
	data = {"grant_type": "client_credentials"}
	result = post(url, headers=headers, data=data)
	json_result = json.loads(result.content)
	token = json_result['access_token']
	return token

def get_auth_header(token):
	return {"Authorization": "Bearer " + token}


## <u>Fonction pour récupérer les Audio features d'une track ou d'une liste de tracks </U>

In [9]:
def get_audio_features(token, track_id):
	url = f'https://api.spotify.com/v1/audio-features/{track_id}'
	headers = get_auth_header(token)
	result = get(url, headers=headers)
	# print(result.headers)
	json_result = json.loads(result.content)
	return json_result


In [10]:
def get_list_features(token, list):
	url = f'https://api.spotify.com/v1/audio-features?ids={list}'
	headers = get_auth_header(token)
	result = get(url, headers=headers)
	# print(result.headers)
	json_result = json.loads(result.content)
	return json_result

## <u>Récupération des données de tracks du df_chart </U>

In [11]:
tracks = df[['track_ID']].drop_duplicates(subset=['track_ID'])
tracks

,track_ID
0,6tNQ70jh4OwmPGpYy6R2o9
1,7AYmToZ5y67fBjq4RLFbuq
2,3w0w2T288dec0mgeZZqoNN
3,1BxfuPKGuaTgP7aM0Bbdwr
4,3rUGC1vUpkDG9CZFHMur1t
...,...
31184,5gEUDNQvoQjdjklrwPdGwD
31192,6FZDfxM3a3UCqtzo5pxSLZ
31194,26w9NTiE9NGjW1ZvIOd1So
31195,4umIPjkehX1r7uhmGvXiSV


In [104]:
token = get_token()

for i in range(0,2700,100):
    list_id = tracks.iloc[i:i+100, 0].tolist()
    list_id = ','.join(list_id)
    result = get_list_features(token, list_id)
    df_tracks = pd.json_normalize(result['audio_features'])
    sleep(random.randint(5,10))

    if i == 0:
        df_tracks.to_csv('Global_tracks.csv', mode = 'w', index=False)
    else:
        df_tracks.to_csv('Global_tracks.csv', mode = 'a', header=False, index=False)

In [12]:
pd.read_csv('Global_tracks.csv')

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.472,0.471,10,-5.692,1,0.0603,0.151,0.000000,0.1400,0.219,105.029,audio_features,6tNQ70jh4OwmPGpYy6R2o9,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,https://api.spotify.com/v1/tracks/6tNQ70jh4Owm...,https://api.spotify.com/v1/audio-analysis/6tNQ...,180304,3
1,0.725,0.709,2,-6.514,1,0.0720,0.588,0.000000,0.1350,0.353,110.024,audio_features,7AYmToZ5y67fBjq4RLFbuq,spotify:track:7AYmToZ5y67fBjq4RLFbuq,https://api.spotify.com/v1/tracks/7AYmToZ5y67f...,https://api.spotify.com/v1/audio-analysis/7AYm...,235636,4
2,0.594,0.811,1,-5.746,1,0.1590,0.189,0.000000,0.3390,0.311,148.144,audio_features,3w0w2T288dec0mgeZZqoNN,spotify:track:3w0w2T288dec0mgeZZqoNN,https://api.spotify.com/v1/tracks/3w0w2T288dec...,https://api.spotify.com/v1/audio-analysis/3w0w...,264324,4
3,0.552,0.702,9,-5.707,1,0.1570,0.117,0.000021,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4
4,0.750,0.733,6,-3.180,0,0.0319,0.256,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,0.761,0.296,0,-10.220,1,0.0484,0.551,0.000492,0.1340,0.579,127.938,audio_features,5gEUDNQvoQjdjklrwPdGwD,spotify:track:5gEUDNQvoQjdjklrwPdGwD,https://api.spotify.com/v1/tracks/5gEUDNQvoQjd...,https://api.spotify.com/v1/audio-analysis/5gEU...,167867,4
2673,0.752,0.488,6,-7.050,1,0.0705,0.297,0.000009,0.0936,0.533,136.041,audio_features,6FZDfxM3a3UCqtzo5pxSLZ,spotify:track:6FZDfxM3a3UCqtzo5pxSLZ,https://api.spotify.com/v1/tracks/6FZDfxM3a3UC...,https://api.spotify.com/v1/audio-analysis/6FZD...,201661,4
2674,0.883,0.600,7,-7.188,1,0.3700,0.114,0.000008,0.0931,0.691,113.903,audio_features,26w9NTiE9NGjW1ZvIOd1So,spotify:track:26w9NTiE9NGjW1ZvIOd1So,https://api.spotify.com/v1/tracks/26w9NTiE9NGj...,https://api.spotify.com/v1/audio-analysis/26w9...,156430,4
2675,0.806,0.546,9,-6.637,1,0.0575,0.300,0.000000,0.1020,0.874,147.986,audio_features,4umIPjkehX1r7uhmGvXiSV,spotify:track:4umIPjkehX1r7uhmGvXiSV,https://api.spotify.com/v1/tracks/4umIPjkehX1r...,https://api.spotify.com/v1/audio-analysis/4umI...,212867,4


## <u>Fonction pour récupérer les données d'un artiste ou d'une liste d'artistes </U>

In [13]:
def get_artist(token, artist_id):
	url = f'https://api.spotify.com/v1/artists/{artist_id}'
	headers = get_auth_header(token)
	result = get(url, headers=headers)
	json_result = json.loads(result.content)
	return json_result

Test:

In [193]:
token = get_token()
artist_json = get_artist(token, '6vWDO969PvNqNYHIOW5v0m')
artist_json

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6vWDO969PvNqNYHIOW5v0m'},
 'followers': {'href': None, 'total': 36395389},
 'genres': ['pop', 'r&b'],
 'href': 'https://api.spotify.com/v1/artists/6vWDO969PvNqNYHIOW5v0m',
 'id': '6vWDO969PvNqNYHIOW5v0m',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb2b7962ada66cbea99dcb0caa',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051742b7962ada66cbea99dcb0caa',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1782b7962ada66cbea99dcb0caa',
   'width': 160}],
 'name': 'Beyoncé',
 'popularity': 87,
 'type': 'artist',
 'uri': 'spotify:artist:6vWDO969PvNqNYHIOW5v0m'}

In [14]:
def get_list_artist(token, list_artist_id):
	url = f'https://api.spotify.com/v1/artists?ids={list_artist_id}'
	headers = get_auth_header(token)
	result = get(url, headers=headers)
	json_result = json.loads(result.content)
	return json_result

## <u>Récupération des données d'artistes (genres, popularity, followers) du df_chart </U>

In [16]:
artist = df[['artist_ID', 'artist']].drop_duplicates(subset=['artist_ID'])
artist

,artist_ID,artist
0,22wbnEMDvgVIAGdFeek6ET,Benson Boone
1,6vWDO969PvNqNYHIOW5v0m,Beyoncé
2,4xPQFgDA5M2xa0ZGo5iIsv,¥$
3,06HL4z0CvFAxyc27GXpf02,Taylor Swift
4,45dkTj5sMRSjrmBSBeiHym,Tate McRae
...,...,...
30981,6i3DxIlAqnDkwELLw4aVrx,Lele Pons
30994,2kCcBybjl3SAtIcwdWpUe3,Lil Peep
30995,1KP6TWI40m7p3QBTU6u2xo,BØRNS
31180,0vRvGUQVUjytro0xpb26bs,Arizona Zervas


In [24]:
token = get_token()

for i in range(0,850,50):
    list_artist_id = ','.join(artist.iloc[i:i+50, 0].tolist())
    result = get_list_artist(token, list_artist_id)
    # if result.get('artists') is not None:

    df_artist = pd.json_normalize(result['artists'])
    df_artist = df_artist.explode('genres')
    sleep(random.randint(5,10))

    if i == 0:
        df_artist.to_csv('Global_artists.csv', mode = 'w', index=False)
    else:
        df_artist.to_csv('Global_artists.csv', mode = 'a', header=False, index=False)
    # else:
    #     print(result)
    #     print(list_artist_id)


In [66]:
dim_artist = pd.read_csv('Vietnam_artists.csv')
dim_artist
dim_artist.groupby('id').first().to_csv('Vietnam_genres.csv', index=True)
pd.read_csv('Vietnam_genres.csv')

,id,genres,href,images,name,popularity,type,uri,external_urls.spotify,followers.href,followers.total
0,00FQb4jTyendYWaN8pK0wa,art pop,https://api.spotify.com/v1/artists/00FQb4jTyen...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Lana Del Rey,90,artist,spotify:artist:00FQb4jTyendYWaN8pK0wa,https://open.spotify.com/artist/00FQb4jTyendYW...,NaN,32720361
1,01XYiBYaoMJcNhPokrg0l0,k-pop,https://api.spotify.com/v1/artists/01XYiBYaoMJ...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",STAYC,62,artist,spotify:artist:01XYiBYaoMJcNhPokrg0l0,https://open.spotify.com/artist/01XYiBYaoMJcNh...,NaN,1745424
2,01veRCzqGIkUMo7L5PyBAD,NaN,https://api.spotify.com/v1/artists/01veRCzqGIk...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kauai45,48,artist,spotify:artist:01veRCzqGIkUMo7L5PyBAD,https://open.spotify.com/artist/01veRCzqGIkUMo...,NaN,27338
3,027TpXKGwdXP7iwbjUSpV8,chicago indie,https://api.spotify.com/v1/artists/027TpXKGwdX...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Walters,71,artist,spotify:artist:027TpXKGwdXP7iwbjUSpV8,https://open.spotify.com/artist/027TpXKGwdXP7i...,NaN,646376
4,048LktY5zMnakWq7PTtFrz,afro r&b,https://api.spotify.com/v1/artists/048LktY5zMn...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",CKay,67,artist,spotify:artist:048LktY5zMnakWq7PTtFrz,https://open.spotify.com/artist/048LktY5zMnakW...,NaN,1419563
...,...,...,...,...,...,...,...,...,...,...,...
511,7tYKF4w9nC0nq9CsPZTHyP,pop,https://api.spotify.com/v1/artists/7tYKF4w9nC0...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",SZA,90,artist,spotify:artist:7tYKF4w9nC0nq9CsPZTHyP,https://open.spotify.com/artist/7tYKF4w9nC0nq9...,NaN,17951273
512,7vk5e3vY1uw9plTHJAMwjN,electro house,https://api.spotify.com/v1/artists/7vk5e3vY1uw...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Alan Walker,80,artist,spotify:artist:7vk5e3vY1uw9plTHJAMwjN,https://open.spotify.com/artist/7vk5e3vY1uw9pl...,NaN,38031538
513,7wqCcGabyuT6f1ZkRxI4ga,NaN,https://api.spotify.com/v1/artists/7wqCcGabyuT...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Võ Hạ Trâm,22,artist,spotify:artist:7wqCcGabyuT6f1ZkRxI4ga,https://open.spotify.com/artist/7wqCcGabyuT6f1...,NaN,6196
514,7yquVKfxBuNFJbG9cy2R8A,v-pop,https://api.spotify.com/v1/artists/7yquVKfxBuN...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Vũ Cát Tường,46,artist,spotify:artist:7yquVKfxBuNFJbG9cy2R8A,https://open.spotify.com/artist/7yquVKfxBuNFJb...,NaN,558092


In [67]:
dim_artist[['id']].drop_duplicates()

,id
0,6HaGTQPmzraVmaVxvz6EUc
1,0aNyWetkjBvIdZvJY00yEa
3,5M3ffmRiOX9Q8Y4jNeR5wu
6,57g2v7gJZepcwsuwssIfZs
10,3JsHnjpbhX4SnySpvpa9DK
...,...
857,7t5H3uQv0Zw6cQUnSTF5BB
859,0ZYv8jRFp5YAC29bfjaP9X
860,64M6ah0SkkRsnPGtGiRAbb
862,2CYz62cthPQ03xzOFpq683


## <u>Test Récupération des genres directement par un endpoint de l'API </U>

In [121]:
def get_genres(token):
	url ='https://api.spotify.com/v1/recommendations/available-genre-seeds'
	headers = get_auth_header(token)
	result = get(url, headers=headers)
	json_result = json.loads(result.content)
	return json_result

token = get_token()
genres = get_genres(token)
genres


{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [140]:
pd.DataFrame.from_dict(genres).to_csv('DIM_GENRES.csv', index=False)

In [141]:
pd.read_csv('DIM_GENRES.csv')

,genres
0,acoustic
1,afrobeat
2,alt-rock
3,alternative
4,ambient
...,...
121,trance
122,trip-hop
123,turkish
124,work-out
